# Interpretation of [BertForSequenceClassification](https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertForSequenceClassification) using [**Captum**](https://captum.ai/)

Source code info:

Used notebook: https://gist.github.com/davidefiocco/3e1a0ed030792230a33c726c61f6b3a5 (adaptation to classification from original tutorial on question answering: https://captum.ai/tutorials/Bert_SQUAD_Interpret)

(Used notebook is based on this github issue: https://github.com/pytorch/captum/issues/303)

Related github issue: https://github.com/pytorch/captum/issues/249

---

Used model: [armheb/DNA_bert_6](https://huggingface.co/armheb/DNA_bert_6?text=The+goal+of+life+is+%5BMASK%5D.)



## Load initial libraries, models, data:

In [1]:
!pip install transformers datasets tokenizers --quiet

In [2]:
!pip install captum

In [3]:
from transformers import BertTokenizer, BertForSequenceClassification

from captum.attr import visualization as viz
from captum.attr import LayerIntegratedGradients

import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

### Load tokenizer:

In [4]:
tokenizer = BertTokenizer.from_pretrained('armheb/DNA_bert_6', num_labels=2)
tokenizer

PreTrainedTokenizer(name_or_path='armheb/DNA_bert_6', vocab_size=4101, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

#### Custom K-mer tokenization:

In [5]:
def kmers(s, k=6):
  return [s[i:i + k] for i in range(0, len(s)-k+1)]

def tokenization(x): 
  return tokenizer(" ".join(kmers(x["seq"])))

example = {'seq': 'ATGGAAAGAGGCACCATTCT'}
print(f'Example: {example}')

example_kmers = " ".join(kmers(example['seq']))
print(f'Example_kmers: {example_kmers}')

tokenized_example = tokenization(example)
print(f'Tokenization example: {tokenized_example}')

decoded_example = tokenizer.decode(tokenized_example['input_ids'])
print(f'Decoded tokenized example: {decoded_example}')

Example: {'seq': 'ATGGAAAGAGGCACCATTCT'}
Example_kmers: ATGGAA TGGAAA GGAAAG GAAAGA AAAGAG AAGAGG AGAGGC GAGGCA AGGCAC GGCACC GCACCA CACCAT ACCATT CCATTC CATTCT
Tokenization example: {'input_ids': [2, 501, 1989, 3848, 3089, 56, 212, 835, 3325, 999, 3983, 3629, 2214, 650, 2587, 2142, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
Decoded tokenized example: [CLS] ATGGAA TGGAAA GGAAAG GAAAGA AAAGAG AAGAGG AGAGGC GAGGCA AGGCAC GGCACC GCACCA CACCAT ACCATT CCATTC CATTCT [SEP]


### Load model:

In [6]:
model = BertForSequenceClassification.from_pretrained('armheb/DNA_bert_6')
model.to(device)

Some weights of the model checkpoint at armheb/DNA_bert_6 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at armheb/DNA_bert_6 and are n

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(4101, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementw

## Get the data:

In [7]:
import pandas as pd

TRAIN_PATH = '/home/jovyan/git_repos/ECCB2022/notebooks/train_set_1_1.txt'
TEST_PATH = '/home/jovyan/git_repos/ECCB2022/notebooks/test_set_1_1.txt'

df_train = pd.read_fwf(TRAIN_PATH, header=None)
df_train.columns = ['seq', 'label']
df_train

,seq,label
0,TCATAGACCCGGTCTCATATGACAAGGAGGGGCATGTCAGACAGTA...,positive
1,NNNNACGAGATCACACAGGTTCTCCGTCGTACACCTCAGTTTTTTC...,positive
2,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNGTCCTACCAGAG...,positive
3,TTTGAATATAAAAAAGTACCTAGAAGAATAAAAGGCACATTCCTTT...,negative
4,NNNNNNNNNNNNNNNGCCTCCCAAACTGCTGGGATTACAGGCGTGA...,negative
...,...,...
299995,CTCTATAAAAGAATTTTAAAATAAGAATTTTCCATGTAAATTAAAG...,negative
299996,NNNNNNNNNNNNNNNNNNCAACAGCAGAGTGTTAAACCAAGTGTGA...,negative
299997,GCCACAGCATTGGAGGAAGGCCTCTCTAAGGCAAGATGTCAGCACT...,negative
299998,NNNNNNNNNNNNNNNACATAAAAGCTACATTTTCAATACAAGATAA...,negative


In [8]:
df_train['label'] = df_train['label'].map(lambda x: 1 if x == 'positive' else 0)
df_train

,seq,label
0,TCATAGACCCGGTCTCATATGACAAGGAGGGGCATGTCAGACAGTA...,1
1,NNNNACGAGATCACACAGGTTCTCCGTCGTACACCTCAGTTTTTTC...,1
2,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNGTCCTACCAGAG...,1
3,TTTGAATATAAAAAAGTACCTAGAAGAATAAAAGGCACATTCCTTT...,0
4,NNNNNNNNNNNNNNNGCCTCCCAAACTGCTGGGATTACAGGCGTGA...,0
...,...,...
299995,CTCTATAAAAGAATTTTAAAATAAGAATTTTCCATGTAAATTAAAG...,0
299996,NNNNNNNNNNNNNNNNNNCAACAGCAGAGTGTTAAACCAAGTGTGA...,0
299997,GCCACAGCATTGGAGGAAGGCCTCTCTAAGGCAAGATGTCAGCACT...,0
299998,NNNNNNNNNNNNNNNACATAAAAGCTACATTTTCAATACAAGATAA...,0


In [9]:
df_test = pd.read_fwf(TEST_PATH, header=None)
df_test.columns = ['seq', 'label']
df_test['label'] = df_test['label'].map(lambda x: 1 if x == 'positive' else 0)
df_test

,seq,label
0,NNNNNNNNNNNATGTTTATTTTGACTATTTACCACTGTTCTTGGTG...,0
1,AACCCGGGTCCCCTGGGTCCGGGGTGGGGTCGGTTAAGGTAGTGGT...,1
2,NNNNTTGTTCTCTTCTTAGTTCCTTGAGGAGCAACATTATGTTATT...,0
3,NNNNNNNNNNNNNNNNNNNNNNNNNNNNTGTGGAGGTACAGATGAG...,1
4,NNNCATGTACCTTATGCATTCAAATTGATGACTTGCTGACTTGTGT...,0
...,...,...
99995,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,0
99996,GATATGGACAATAAGGTCCAGGCTGACGTGGTGTCAGATGGAGATG...,0
99997,CAGCCCTGAGGGCAGGACTTGACCCCAGGCTTCCGGGCACTTACCT...,0
99998,CTTGCGTCACGTATGGCCTCATGTCTGTGCACAGGTCATGGAGACT...,0


In [10]:
from datasets import Dataset
# dataset_train = Dataset.from_pandas(df_train[:10000])
dataset_train = Dataset.from_pandas(df_train)
dataset_train

Dataset({
    features: ['seq', 'label'],
    num_rows: 300000
})

In [11]:
# dataset_test = Dataset.from_pandas(df_test[:10000])
dataset_test = Dataset.from_pandas(df_test)
dataset_test

Dataset({
    features: ['seq', 'label'],
    num_rows: 100000
})

In [12]:
# one training sample sequence and its label
dataset_train[0]

{'seq': 'TCATAGACCCGGTCTCATATGACAAGGAGGGGCATGTCAGACAGTACCGAAGAAAACTGATCCTTTCCCTTTAGGGGGTCGGGGAACACGAAGGACCCACTCTGCTACGGGGTGGGACGAAGTCGGGGGGGAGGTACTCGACATGGGTGATAGGTTGGTCAGGGTTACCCTACTTGAACCATGGAGTCTACCTTTACGTC',
 'label': 1}

In [13]:
from huggingface_hub import notebook_login

notebook_login()

In [14]:
dataset_train_tokenized = dataset_train.map(tokenization, batched=False)
dataset_test_tokenized = dataset_test.map(tokenization, batched=False)
dataset_train_tokenized

  0%|          | 0/300000 [00:00<?, ?ex/s]

  0%|          | 0/100000 [00:00<?, ?ex/s]

Dataset({
    features: ['seq', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 300000
})

In [16]:
dataset_train_tokenized.push_to_hub("g_quad_DNA_tokenized_k6_train")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

/home/jovyan/my-conda-envs/ml_env/lib/python3.8/site-packages/huggingface_hub/hf_api.py:2165: FutureWarning: `identical_ok` has no effect and is deprecated. It will be removed in 0.11.0.
  warnings.warn(


In [17]:
dataset_test_tokenized.push_to_hub("g_quad_DNA_tokenized_k6_test")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
print(dataset_train_tokenized[0]['input_ids'])

[2, 1560, 2129, 311, 1231, 815, 3248, 692, 2754, 2811, 3038, 3947, 3485, 1638, 2441, 1558, 2124, 289, 1143, 461, 1829, 3208, 532, 2113, 248, 980, 3908, 3332, 1027, 4093, 4070, 3980, 3618, 2171, 477, 1896, 3473, 1591, 2253, 808, 3218, 569, 2263, 847, 3376, 1201, 693, 2760, 2833, 3125, 197, 773, 3079, 14, 44, 161, 630, 2507, 1823, 3182, 426, 1690, 2651, 2399, 1391, 1454, 1706, 2714, 2649, 2392, 1364, 1348, 1284, 1028, 4098, 4091, 4064, 3956, 3524, 1796, 3073, 4085, 4039, 3853, 3111, 144, 561, 2229, 712, 2836, 3137, 247, 975, 3887, 3245, 679, 2702, 2603, 2206, 620, 2467, 1662, 2537, 1943, 3664, 2356, 1220, 772, 3074, 4092, 4068, 3972, 3585, 2039, 4048, 3889, 3253, 712, 2834, 3131, 224, 884, 3524, 1796, 3076, 4100, 4100, 4097, 4088, 4052, 3906, 3321, 983, 3918, 3371, 1184, 625, 2487, 1741, 2854, 3212, 548, 2180, 514, 2044, 4065, 3958, 3529, 1816, 3156, 322, 1274, 988, 3940, 3458, 1531, 2013, 3944, 3476, 1604, 2306, 1018, 4057, 3927, 3407, 1327, 1198, 681, 2711, 2638, 2346, 1180, 609, 2421,

## Captum interpretation:

### 1. On untrained model:

*Helper function to perform forward pass of the model and make predictions:*

In [19]:
def predict(inputs):
    score = model(inputs)[0]
    return score

*Custom forward function that will allow us to access the postitions of our prediction using position input argument:*

In [20]:
### original for question answering looked like this: ######
# def squad_pos_forward_func(inputs, token_type_ids=None, position_ids=None, attention_mask=None, position=0):
#     pred = predict(inputs,
#                    token_type_ids=token_type_ids,
#                    position_ids=position_ids,
#                    attention_mask=attention_mask)
#     pred = pred[position]
#     return pred.max(1).values
############################################################

def custom_forward(inputs):
    preds = predict(inputs)
    return torch.softmax(preds, dim = 1)[0][0].unsqueeze(-1)

Compute attributions with respect to the `BertEmbeddings` layer:

1. define baselines/ references,
2. numericalize baselines and inputs.

*(helper functions to achieve that)*

In [21]:
ref_token_id = tokenizer.pad_token_id  # token used for generating token reference
sep_token_id = tokenizer.sep_token_id  # token added to the end of the input text
cls_token_id = tokenizer.cls_token_id  # token used at the beginning of the input text
print(ref_token_id)
print(sep_token_id)
print(cls_token_id)

0
3
2


In [22]:
def construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id):
    text_ids = tokenizer.encode(text, add_special_tokens=False)
    # construct input token ids
    input_ids = [cls_token_id] + text_ids + [sep_token_id]
    # construct reference token ids 
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(text_ids) + [sep_token_id]

    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device), len(text_ids)

Define the input sequence `sample1` that we'd like to use as an input for our Bert model and interpret what the model was focusing on when predicting the class:

In [23]:
def get_sample_and_label(dataset, seq_id):
    seq = dataset[seq_id]['seq']
    label = dataset[seq_id]['label']
    seq_kmers = kmers(seq)
    tokenized_seq = tokenizer.encode(seq_kmers, add_special_tokens=False)
    print(f'Seq label: {label}')
    print(f'Input seq (raw): {seq}')
    print(f'Returned (tokenized[:10]): {tokenized_seq[:10]}')
    return tokenized_seq, label

sample1, sample_label1 = get_sample_and_label(dataset_test, 0)

Seq label: 0
Input seq (raw): NNNNNNNNNNNATGTTTATTTTGACTATTTACCACTGTTCTTGGTGTTTTTATGGAGAAAAATGTTTTCAGAGGTTTTTATTCTGCCAGTTGGAGGTACTTCTCTGTAACTGACTTCCGTAGTTAAGTGAAGGACTTATTCCCAAAAACCTGATTTTAGCTCAGGTTTTATAAATTNNNNNNNNNNNNNNNNNNNNNNNN
Returned (tokenized[:10]): [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


Let's numericalize the input `sample1` and generate corresponding baselines/references for all three sub-embeddings (word, token type and position embeddings) types using our helper functions defined above:

In [24]:
input_ids, ref_input_ids, sep_id = construct_input_ref_pair(sample1, ref_token_id, sep_token_id, cls_token_id)
print(input_ids)
print(ref_input_ids)
print(sep_id)

tensor([[   2,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          474, 1881, 3414, 1354, 1306, 1114,  348, 1377, 1399, 1486, 1833, 3222,
          586, 2330, 1113,  343, 1359, 1325, 1191,  654, 2604, 2210,  634, 2523,
         1886, 3434, 1436, 1636, 2434, 1532, 2018, 3962, 3546, 1882, 3418, 1369,
         1366, 1356, 1316, 1153,  504, 2001, 3893, 3269,  773, 3077,    6,   12,
           34,  122,  474, 1882, 3419, 1373, 1384, 1425, 1592, 2260,  834, 3322,
          986, 3930, 3418, 1369, 1366, 1354, 1307, 1118,  364, 1443, 1663, 2541,
         1960, 3730, 2618, 2268,  868, 3457, 1528, 2004, 3906, 3321,  983, 3918,
         3370, 1179,  606, 2411, 1438, 1644, 2466, 1657, 2517, 1863, 3342, 1068,
          161,  631, 2510, 1834, 3227,  607, 2416, 1458, 1721, 2776, 2898, 3386,
         1241,  853, 3400, 1298, 1084,  225,  885, 3528, 1812, 3137,  247,  974,
         3882, 3225,  598, 2378, 1307, 1119,  367, 1453, 1701, 2693, 2565, 2053,
            7,   15,   46,  

In [25]:
model(input_ids)

SequenceClassifierOutput(loss=None, logits=tensor([[0.1872, 0.2083]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [26]:
predict(input_ids)

tensor([[0.1872, 0.2083]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [27]:
custom_forward(input_ids)

tensor([0.4947], device='cuda:0', grad_fn=<UnsqueezeBackward0>)

There are 2 different ways of computing the attributions for emebdding layers. One option is to use `LayerIntegratedGradients` and compute the attributions with respect to `BertEmbedding`. The second option is to use `LayerIntegratedGradients` for each `word_embeddings`, `token_type_embeddings` and `position_embeddings` and compute the attributions w.r.t each embedding vector.

In [28]:
lig = LayerIntegratedGradients(custom_forward, model.bert.embeddings)

Helper function to summarize attributions for each word token in the sequence:

In [29]:
def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    return attributions

In [30]:
def interpret_and_visualize(tokenized_sample, label):
    input_ids, ref_input_ids, sep_id = construct_input_ref_pair(tokenized_sample, ref_token_id, sep_token_id, cls_token_id)
    score = predict(input_ids)

    attributions, delta = lig.attribute(inputs=input_ids,
                                    baselines=ref_input_ids,
                                    return_convergence_delta=True)
    
    indices = input_ids[0].detach().tolist()
    
    all_tokens = tokenizer.convert_ids_to_tokens(indices)
    
    attributions_sum = summarize_attributions(attributions)

    score_vis = viz.VisualizationDataRecord(
        word_attributions=attributions_sum, 
        pred_prob=torch.softmax(score, dim=1)[0][0],
        pred_class=torch.argmax(torch.softmax(score, dim=1)[0]), 
        true_class=label, 
        attr_class=tokenized_sample, 
        attr_score=attributions_sum.sum(), 
        raw_input_ids=all_tokens, 
        convergence_score=delta)

    print('\033[1m', 'Visualization For Score', '\033[0m')
    viz.visualize_text([score_vis])

In [31]:
interpret_and_visualize(sample1, sample_label1)

 Visualization For Score 


### 2. On trained model:

#### Train the model:

In [33]:
from transformers import TrainingArguments, Trainer
from datasets import load_metric
import numpy as np

BATCH_SIZE = 32
LEARNING_RATE = 1e-5
EPOCHS = 20

training_arguments = TrainingArguments(
        output_dir='outputs', 
        learning_rate=LEARNING_RATE, 
        num_train_epochs=EPOCHS, 
        evaluation_strategy="epoch", 
        logging_strategy='epoch',
        per_device_train_batch_size=BATCH_SIZE, 
        per_device_eval_batch_size=BATCH_SIZE,
        fp16=True,
    )

def compute_metrics(eval_preds):
    metric = load_metric("accuracy")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=dataset_train_tokenized,
    eval_dataset=dataset_test_tokenized,
    compute_metrics=compute_metrics,
)
trainer

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using cuda_amp half precision backend


In [34]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: seq. If seq are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/home/jovyan/my-conda-envs/ml_env/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 300000
  Num Epochs = 20
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 187500
COMET WARNING: Comet has disabled auto-logging functionality as it has been imported after the following ML modules: torch. Metrics and hyperparameters can sti

Epoch,Training Loss,Validation Loss,Accuracy
1,0.093100,0.085739,0.970770
2,0.068700,0.075620,0.974370
3,0.061800,0.090386,0.971890
4,0.055900,0.077345,0.975900
5,0.050300,0.089248,0.976480
6,0.045300,0.093723,0.975520
7,0.039900,0.098384,0.975040
8,0.035200,0.111403,0.974490
9,0.031100,0.124405,0.974370
10,0.027500,0.126049,0.973660


Saving model checkpoint to outputs/checkpoint-500
Configuration saved in outputs/checkpoint-500/config.json
Model weights saved in outputs/checkpoint-500/pytorch_model.bin
Saving model checkpoint to outputs/checkpoint-1000
Configuration saved in outputs/checkpoint-1000/config.json
Model weights saved in outputs/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to outputs/checkpoint-1500
Configuration saved in outputs/checkpoint-1500/config.json
Model weights saved in outputs/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to outputs/checkpoint-2000
Configuration saved in outputs/checkpoint-2000/config.json
Model weights saved in outputs/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to outputs/checkpoint-2500
Configuration saved in outputs/checkpoint-2500/config.json
Model weights saved in outputs/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to outputs/checkpoint-3000
Configuration saved in outputs/checkpoint-3000/config.json
Model weights save

TrainOutput(global_step=187500, training_loss=0.03208499766031901, metrics={'train_runtime': 29810.5894, 'train_samples_per_second': 201.271, 'train_steps_per_second': 6.29, 'total_flos': 6.0741653796e+17, 'train_loss': 0.03208499766031901, 'epoch': 20.0})

In [ ]:
model.save_pretrained("DNABert_k6_G_quad")

Configuration saved in DNABert_k6_G_quad/config.json
Model weights saved in DNABert_k6_G_quad/pytorch_model.bin


### Interpret again

In [37]:
interpret_and_visualize(sample1, sample_label1)

 Visualization For Score 


## Other input sequences:

### Test data:

In [38]:
for i in range(20):
    sample, sample_label = get_sample_and_label(dataset_test, i)
    interpret_and_visualize(sample, sample_label)

Seq label: 0
Input seq (raw): NNNNNNNNNNNATGTTTATTTTGACTATTTACCACTGTTCTTGGTGTTTTTATGGAGAAAAATGTTTTCAGAGGTTTTTATTCTGCCAGTTGGAGGTACTTCTCTGTAACTGACTTCCGTAGTTAAGTGAAGGACTTATTCCCAAAAACCTGATTTTAGCTCAGGTTTTATAAATTNNNNNNNNNNNNNNNNNNNNNNNN
Returned (tokenized[:10]): [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 Visualization For Score 


Seq label: 1
Input seq (raw): AACCCGGGTCCCCTGGGTCCGGGGTGGGGTCGGTTAAGGTAGTGGTTCAAGAAACTGGAGGTGAGGACTGTAACTTGGTCTAGGGACGAGAAGGAGGACGATAGGGGTAACAGGGGACAGGGTCGGTGAGAGTCGTGTATGGACCCAGTAATCTGTCTAGGACAGAGGGCGGAGAGGACGGAGGGGACGACCGGAACAAT
Returned (tokenized[:10]): [176, 692, 2756, 2818, 3067, 4063, 3951, 3503, 1710, 2732]
 Visualization For Score 


Seq label: 0
Input seq (raw): NNNNTTGTTCTCTTCTTAGTTCCTTGAGGAGCAACATTATGTTATTTGAGCTCTTTCTTCTTTTTGATATAGGCATTTATTGCTGCAAACATTGATCTTACTGCTTTTGCTGTATTTCATAAGCTTTTGGTATGTTGTGTTTCCATTTTTGTTTNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
Returned (tokenized[:10]): [1, 1, 1, 1, 1499, 1886, 3435, 1438, 1642, 2459]
 Visualization For Score 


Seq label: 1
Input seq (raw): NNNNNNNNNNNNNNNNNNNNNNNNNNNNTGTGGAGGTACAGATGAGAGTGAGGTGGGCATGGGATCAAGCCCTGTAGGGTTCTGGCATTTAGAGGTAGAAGAGAGGAGGACAGGGTGGCAGGTGGAGGAGGCCCAGGGGGGATGGATCCCTGGAACCCAGAGCAGTTCCAGCTGCTAANNNNNNNNNNNNNNNNNNNNNN
Returned (tokenized[:10]): [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 Visualization For Score 


Seq label: 0
Input seq (raw): NNNCATGTACCTTATGCATTCAAATTGATGACTTGCTGACTTGTGTCAATGTTCTATTTTGGAAATACTTGCAAAATTGCGCTTATGATCTTGTATTCCAATGATTTACTTACATCACTATCTCCATCCCATGACTGTGTGCCCTGGAGAGCANNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
Returned (tokenized[:10]): [1, 1, 1, 2169, 471, 1871, 3374, 1194, 665, 2646]
 Visualization For Score 


...

Seq label: 0
Input seq (raw): CTACTGAAAAAGACAATTGATTTCTCCTCTCATTCCCTTGGAAAAGTCAGTTTTTATCCATTTTAAAGTATATTTTAGCATTCCTGATGGTATGTGTATATGCACACATATGTATAGATATATAAATATATACATGTTTTAATTCTCCTCTGAATTGGTTGCAACATCTTATTGGTCATCTCATTAGTGAGTTCAGTAAA
Returned (tokenized[:10]): [2348, 1185, 629, 2501, 1797, 3077, 8, 17, 55, 205]
 Visualization For Score 


Seq label: 0
Input seq (raw): CAAGAATGAAGGATGCTTAGCAGGTGTTGAAGAATCCAAACTTAGAATTGATGCTTCTTAAAGAGGGCTTCTGGGAGCTTAGATCAAGTGGCCATTATCTGGCCCAGCCACCAGCCAATGGGGCTGGAAAGATGGAATGAAAGTTCTGGATTCCCAATCTCTTTGTACTCATCTATATTATTGCACTAATAAAGCTTATT
Returned (tokenized[:10]): [2101, 198, 780, 3105, 117, 456, 1812, 3137, 246, 972]
 Visualization For Score 


Seq label: 0
Input seq (raw): TCTAAAGTTAAGGCTTCAAAACTACATTCCTGAAATTCAAGTTGGCTCAAACAATATCCAACCTCATTTTGCGCAGTCAAAGAAAACTGCTGTTCTTTACTATTGGCTAAAGACATCAAGCCACCACCCTCCTCCAATCTTCAGAGCAATTCATTTGATTCTCAAAATTAGACCTCTGCAAATTTTAAAATTATTTATCA
Returned (tokenized[:10]): [1605, 2312, 1042, 58, 217, 853, 3400, 1300, 1091, 254]
 Visualization For Score 


Seq label: 0
Input seq (raw): NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNTTCTAAACTTTGCAAAGTTTTTTTCTTTTAAAAGATTGTTCAATTCTTACCATTGACCTGTGTATATGCCATATCAAGCTATTTAATGACTAGACTTTCATATTTNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
Returned (tokenized[:10]): [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 Visualization For Score 


Seq label: 0
Input seq (raw): CTGAATTGGGTTGAATGGTGGCCTCCCAAAACATATATCTATTTGGAACCTGTAATGGTAACCTTATTTGGAGAAAGGATCTTAGCAAATGTAATTAAGAATCCCAAGATGACATCATCCTGGATAATCCTGGTGAATCCTAAATCCAGTTGCAAGTATCCTTATAAAAGAAAGTTCAGGGCTATATTAGACACATAGAC
Returned (tokenized[:10]): [2502, 1802, 3100, 100, 388, 1538, 2042, 4060, 3937, 3445]
 Visualization For Score 


### Train data:

In [39]:
for i in range(20):
    sample, sample_label = get_sample_and_label(dataset_train, i)
    interpret_and_visualize(sample, sample_label)

Seq label: 1
Input seq (raw): TCATAGACCCGGTCTCATATGACAAGGAGGGGCATGTCAGACAGTACCGAAGAAAACTGATCCTTTCCCTTTAGGGGGTCGGGGAACACGAAGGACCCACTCTGCTACGGGGTGGGACGAAGTCGGGGGGGAGGTACTCGACATGGGTGATAGGTTGGTCAGGGTTACCCTACTTGAACCATGGAGTCTACCTTTACGTC
Returned (tokenized[:10]): [1560, 2129, 311, 1231, 815, 3248, 692, 2754, 2811, 3038]
 Visualization For Score 


Seq label: 1
Input seq (raw): NNNNACGAGATCACACAGGTTCTCCGTCGTACACCTCAGTTTTTTCTAATAAGCCGGCCCGCGCCACCGAGTGCGGACATTAGGGTCGTGAAACCCTCCGGCTCCGCCCGCCTAGTGCTCCAGTCCTCTAGCTCTGGTAGGGCCGATTTTGCCACTTTGGGGCATAGATGATTTTTATGTTTTTTAATCGGCCCGCATCN
Returned (tokenized[:10]): [1, 1, 1, 1, 721, 2870, 3275, 797, 3175, 397]
 Visualization For Score 


Seq label: 1
Input seq (raw): NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNGTCCTACCAGAGTTACAGGACTGGAGCACTAGACGGGCGGAGCCGGAGAGTTTCACGACCCTAATGTCCGCACTCGGTGGTGGGGGCCGGGGTCCTGTGTGTCGAAATTTTAAAGAGGAACCAGAGTGGGTCACGGTTGGTGGATTTTGGNNNNNNNNNNNNNNNN
Returned (tokenized[:10]): [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 Visualization For Score 


Seq label: 0
Input seq (raw): TTTGAATATAAAAAAGTACCTAGAAGAATAAAAGGCACATTCCTTTATGTATCTTATTAGGCTTTCTTTTTTTCCTAAGTAGTAAAACAATCATTAGTTTCACTGAGTATTTTGGACAGCATATTAGAAATTTGAAGTACCAAGTTTACTATTCTTTACTGGAGATTGTCCAGCAGAGGAAAGCAAATACCAGTGGATGT
Returned (tokenized[:10]): [1397, 1478, 1801, 3094, 73, 277, 1093, 261, 1029, 5]
 Visualization For Score 


Seq label: 0
Input seq (raw): NNNNNNNNNNNNNNNGCCTCCCAAACTGCTGGGATTACAGGCGTGAGCCACTGCGCCCGACTGGAAGGGGGCTTTCTTTAGAAAATAGGTAAATGTTAGTAGGGTGGAAACCAACCCTGGCCCCTTTCAAATGTGTGGTTTTTTTTTTTTTTTTTGGAGACAGTTGTACTCTTTTCTCCCNNNNNNNNNNNNNNNNNNNN
Returned (tokenized[:10]): [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 Visualization For Score 


...

Seq label: 1
Input seq (raw): CGATTAATAATTCCGCTCTCACCGTCAAGATACAAGGGAGGGGGGGTTTTTTTCCTTCCTTTTCTCCTTTTGATTCAAAGTCACGATTTTATAGAACACCGGGACCCATACGTCCTAACGTATAGACTGGTATTGATCGGATCGTCGCTCAAGGGATGCCTATCGATTTTATGTAGTAAAACTCCACCTAATCGATGGGG
Returned (tokenized[:10]): [2841, 3157, 326, 1289, 1045, 70, 266, 1051, 95, 368]
 Visualization For Score 


Seq label: 1
Input seq (raw): CCAAACTTAGCGCCTTAAGTCAAACACCATTTTATTATATCTCACAATGTTGCATGTCAGGAATTTGGGGCAGGACCCAGCTGGGAGATGCTTTGCAATAGGATGGGGCCCTGGGGGGTGAGTAGGGAGGGGGCTGTGTGTGTGTCGAGGTCATTTGACAATAAAGGCCCTTTTTGCTCTAGTATTCTAAGATTCTTCAA
Returned (tokenized[:10]): [2567, 2062, 42, 153, 600, 2387, 1344, 1267, 959, 3822]
 Visualization For Score 


Seq label: 1
Input seq (raw): NNNNNNNNNNNNNNNNNNAATATGCCATGCATAGATGGAAAAATATCGCATGATCTCACTTATATGTGAAATCTAAAAAAAACAAAAGGCCGAATTGAGGAGACAGAAAATAAAACAGTGGTTACCAGAATTGGGGTGGGGGTGGGAGGAAATGGGGTGACATAGACCAAAGGATACACAGTAGCAAATACATAGGACNN
Returned (tokenized[:10]): [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 Visualization For Score 


Seq label: 0
Input seq (raw): GGCCTTCTCTGCATCTATTGAGATAATCATGTGGTTTTTGTCTTTGGCTCTGTTTATATGCTGGATTACACTTATTGATTTGCCTATATTGAACCAGCCTTGCATCCCAGGGATGAAGCCCACTTGATCATGGTGGATAAGCTTTTTGATGTGCTGCTGGATTCGTTTTGCCAGTATTTTATTGAGGATTTTTGCATCAA
Returned (tokenized[:10]): [4010, 3739, 2654, 2411, 1438, 1644, 2467, 1661, 2534, 1931]
 Visualization For Score 


Seq label: 0
Input seq (raw): NNNCTAGAGCAGAGAATTTTAAGACGGTGATCTCTCCTGCTTTCAGTTCCATTTTGGCACAAGTCTTATGTCAACATCTCACCTCCCTTTGATGCAATGATATACACTTTTTCCAATTTAAGAAGGAGTCTGGACTCTGTGGTGTCCTTTCCTTGGTCTATCAAGATCATCCCTTAGGAAGCAGAGCTGAATTCTGGANN
Returned (tokenized[:10]): [1, 1, 1, 2360, 1235, 829, 3304, 913, 3640, 2257]
 Visualization For Score 


*Colours: https://github.com/pytorch/captum/issues/249#issuecomment-580569266*

*That output is the prediction probability (p) of being a positive sentiment. A negative sentiment would be (1 - p).
In our case we attribute positive sentiment probability (p) to the inputs of our model and in case something is predicted with high probability as positive sentiment we see many tokens that contribute positively to the positive sentiment.*

*In case when p is very low, there are no words contributing to the positive sentiment and when we attribute to the positive sentiment prob (p) we find words that pull away (influence negatively) to the positive sentiment. Those tokens are obviously the ones that pull towards the negative sentiment with higher (1-p) probability.*

https://github.com/pytorch/captum/issues/249#issuecomment-580846266

*In a general case, red means that those tokens are pulling away from the Hate Speech (1) and most probably pulling towards the opposite class however I think that red might not always mean that it will always attribute to the other class. I think that's the assumption that we make here. We assume that the classifier is able to identify that a token is negatively correlated with the Hate Speech(1) class so it must know something about that token, namely, that it is strongly pulling towards the opposite class (because there are no other options) and this is much easier to imagine for 2 class problem.*